In [ ]:
import pandas as pd
import FWI
import matplotlib.pyplot as plt
import numpy as np
import os
import cfgrib
import xarray as xrs
import math
import calendar

def DistanceTwoPoints(lat1, lon1, lat2, lon2):
    R = 6371e3  # Radius of the Earth in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

def find_nearest(lat, lon, latitudes, longitudes):
    min_distance = None
    nearest_point = None

    for lat2, lon2 in zip(latitudes, longitudes):
        distance = DistanceTwoPoints(lat, lon, lat2, lon2)
        if min_distance is None or distance < min_distance:
            min_distance = distance
            nearest_point = (lat2, lon2)

    return min_distance, nearest_point

def find_nearest_index(lat, lon, latitudes, longitudes):
    distances = np.array([DistanceTwoPoints(lat, lon, lat2, lon2) for lat2, lon2 in zip(latitudes, longitudes)])
    sorted_distances_indices = np.argsort(distances)
    nearest_points = [(latitudes[i], longitudes[i]) for i in sorted_distances_indices[:4]]
    min_distances = distances[sorted_distances_indices[:4]]
    return min_distances, nearest_points

def convert_longitude360(lon):
    if lon < 0:
        return lon + 360
    return lon

def convert_longitude(lon):
    if lon > 180:
        return lon - 360
    return lon


def total_precipitation(initial_date, final_date, dmet):
    dmet['hourly.time'] = pd.to_datetime(dmet['hourly.time'])
    dmet_copy = dmet.copy()
    dmet_copy.set_index('hourly.time', inplace=True)
    selected_rows = dmet_copy.loc[initial_date:final_date]
    total_precipitation = selected_rows['hourly.precipitation'].sum()

    return total_precipitation

In [ ]:
import re

def sort_key(filename):
    match = re.search(r'(\d+)', filename)
    if match:
        return int(match.group(1))
    else:
        return 0

fogos = [f for f in os.listdir("OUTPUTNOFIRE") if f.endswith('.csv')]

fogos = sorted(fogos, key=sort_key)

print(len(fogos))
print(fogos)

In [ ]:
dfCur = pd.read_csv('OUTPUT/' + fogos[0])
print(dfCur.head())
dfInd = pd.read_csv('./NaturalFires/NaturalFires.csv')
print(dfInd.__len__())

print(dfInd.loc[167])

In [ ]:
from FWI import FFMC


def sort_key(filename):
    match = re.search(r'(\d+)', filename)
    if match:
        return int(match.group(1))
    else:
        return 0

fogos = [f for f in os.listdir("OUTPUTNOFIRE") if f.endswith('.csv')]

fogos = sorted(fogos, key=sort_key)

print(len(fogos))
print(fogos)

indexes = [1082,1104,1119,1122,1123,1124,1126,1129,1132,1221,1447,1588]

with open("coordinates.txt", "r") as f:
    coords = f.readlines()

for j in range(1711, len(fogos)):
    start_index = 24+12

    mts = {
            1: 31,
            2: 28,
            3: 31,
            4: 30,
            5: 31,
            6: 30,
            7: 31,
            8:31, 
            9:30, 
            10:31,
            11:30,
            12:31}


    #dfInd = pd.read_csv('./NaturalFires/NaturalFires.csv')

    row = coords[0].strip().split(',')
    latitude_R = row[1]
    longitude_R = row[2]

    dfCurr = pd.read_csv("OUTPUTNOFIRE/" + fogos[j])
    hora = dfCurr['hourly.time'].iloc[start_index]
    print(hora)
    yearInic = hora.split('-')[0]
    print(yearInic)

    
    ds = xrs.open_dataset(f"FWI_HISTORY/{yearInic}.grib", engine='cfgrib')
    print(ds)

    fwinx_at_time = ds['fwinx'].sel(time=f'{hora}')

    print(fwinx_at_time)

    latitudes = fwinx_at_time.latitude
    longitudes = fwinx_at_time.longitude

    print(latitudes.values)
    print(len(latitudes.values))
    print(longitudes.values)
    print(len(longitudes.values))

    novas_lat = []
    novas_lon = []
    for lat in range(len(latitudes)):
        for lon in range(len(longitudes)):
            novas_lat.append(latitudes[lat].values)
            novas_lon.append(longitudes[lon].values)


    LATITUDE_SING = float(latitude_R)
    LONGITUDE_SING = float(longitude_R)
    min_distance, nearest_point = find_nearest(LATITUDE_SING, convert_longitude360(LONGITUDE_SING), novas_lat, novas_lon)
    print(min_distance, nearest_point)

    print("Latitude: ", nearest_point[0])
    print("Longitude: ", nearest_point[1])

    fwinx_value = ds['fwinx'].sel(time=f'{hora}', latitude=nearest_point[0], longitude=nearest_point[1])
    drtcode_value = ds['drtcode'].sel(time=f'{hora}', latitude=nearest_point[0], longitude=nearest_point[1])
    dufmcode_value = ds['dufmcode'].sel(time=f'{hora}', latitude=nearest_point[0], longitude=nearest_point[1])
    ffmcode_value = ds['ffmcode'].sel(time=f'{hora}', latitude=nearest_point[0], longitude=nearest_point[1])

    
    LAT = nearest_point[0]
    FFMCPrev = ffmcode_value.values.item()
    DMCPrev = dufmcode_value.values.item()
    DCPrev = drtcode_value.values.item()

    fwix_values = [None] * len(dfCurr)
    ffmc_values = [None] * len(dfCurr)
    dmc_values = [None] * len(dfCurr)
    dc_values = [None] * len(dfCurr)
    isi_values = [None] * len(dfCurr)
    bui_values = [None] * len(dfCurr)

    

    dfCurr['hourly.time'] = pd.to_datetime(dfCurr['hourly.time'])

    min_date = dfCurr['hourly.time'].min()
    max_date = dfCurr['hourly.time'].max()

    for i in range(start_index, len(dfCurr)):
            MONTH = dfCurr.iloc[i]['hourly.time'].month
            DAY = dfCurr.iloc[i]['hourly.time'].day
            HOUR = dfCurr.iloc[i]['hourly.time'].hour
            year = dfCurr.iloc[i]['hourly.time'].year

            print(f"$-->{j}; ", MONTH, DAY, HOUR, year)

            TEMP = dfCurr.iloc[i]['hourly.temperature_2m']
            RH = dfCurr.iloc[i]['hourly.relative_humidity_2m']
            WIND = dfCurr.iloc[i]['hourly.wind_speed_10m']


            if(calendar.isleap(int(year))):
                mts[2] = 29
            else:
                mts[2] = 28

            if(int(HOUR) < 10):
                HOUR = '0' + str(HOUR)


            if(int(DAY) == 1):
                if(int(MONTH) == 1):
                    LAST_DAY = mts[12]
                    LAST_MONTH = '12'
                    initial_date = f'{str(int(year) - 1)}-{LAST_MONTH}-{LAST_DAY}T{HOUR}:00'
                    final_date = f'{year}-{MONTH}-{DAY}T{HOUR}:00'
                else:
                    LAST_DAY = mts[int(MONTH) - 1]
                    LAST_MONTH = str(int(MONTH) - 1)

                    if(int(LAST_MONTH) < 10):
                        LAST_MONTH = '0' + str(int(LAST_MONTH))
                        
                    initial_date = f'{year}-{LAST_MONTH}-{LAST_DAY}T{HOUR}:00'
                    final_date = f'{year}-{MONTH}-{DAY}T{HOUR}:00'

            elif((int(DAY) < 10) and (int(DAY) > 1)):
                id = '0' + str(int(DAY)-1)
                fd = '0' + str(int(DAY))

                initial_date = f'{year}-{MONTH}-{id}T{HOUR}:00'
                final_date = f'{year}-{MONTH}-{fd}T{HOUR}:00'
            else:
                id = str(int(DAY)-1)
                fd = str(int(DAY))

                initial_date = f'{year}-{MONTH}-{id}T{HOUR}:00'
                final_date = f'{year}-{MONTH}-{fd}T{HOUR}:00'

        
            print("Initial Date: ", initial_date)
            print("Final Date: ", final_date)

            initial_date_dt = pd.to_datetime(initial_date)
            final_date_dt = pd.to_datetime(final_date)

            

            if min_date <= initial_date_dt <= max_date:
                if(int(HOUR) == 12):
                    print("Este caso")
                    hora_reset = dfCurr.iloc[i]['hourly.time']

                    print(f"year: {yearInic}; hora_reset: {hora_reset.year}")

                    if(int(hora_reset.year) != int(yearInic)):
                        print("Ano novo aberto")
                        ds = xrs.open_dataset(f"FWI_HISTORY/{hora_reset.year}.grib", engine='cfgrib')
                        print(ds)
                        novas_lat = []
                        novas_lon = []
                        for lat in range(len(latitudes)):
                            for lon in range(len(longitudes)):
                                novas_lat.append(latitudes[lat].values)
                                novas_lon.append(longitudes[lon].values)


                        LATITUDE_SING = float(latitude_R)
                        LONGITUDE_SING = float(longitude_R)
                        min_distance, nearest_point = find_nearest(LATITUDE_SING, convert_longitude360(LONGITUDE_SING), novas_lat, novas_lon)
                        print(min_distance, nearest_point)


                    
                    fwinx_value = ds['fwinx'].sel(time=f'{hora_reset}', latitude=nearest_point[0], longitude=nearest_point[1])
                    drtcode_value = ds['drtcode'].sel(time=f'{hora_reset}', latitude=nearest_point[0], longitude=nearest_point[1])
                    dufmcode_value = ds['dufmcode'].sel(time=f'{hora_reset}', latitude=nearest_point[0], longitude=nearest_point[1])
                    ffmcode_value = ds['ffmcode'].sel(time=f'{hora_reset}', latitude=nearest_point[0], longitude=nearest_point[1])
                    bui_value = ds['fbupinx'].sel(time=f'{hora_reset}', latitude=nearest_point[0], longitude=nearest_point[1])
                    isi_value = ds['infsinx'].sel(time=f'{hora_reset}', latitude=nearest_point[0], longitude=nearest_point[1])

                    FFMCPrev = ffmcode_value.values.item()
                    DMCPrev = dufmcode_value.values.item()
                    DCPrev = drtcode_value.values.item()

                    fwix_values[i] = fwinx_value.values.item()
                    ffmc_values[i] = FFMCPrev
                    dmc_values[i] = DMCPrev
                    dc_values[i] = DCPrev
                    isi_values[i] = isi_value.values.item()
                    bui_values[i] = bui_value.values.item()

                    

                else:
                    RAIN = total_precipitation(initial_date, final_date, dfCurr)

                    try:
                        if(FFMCPrev == 0):
                            FFMCPrev = 0.000000001

                        if(DMCPrev == 0):
                            DMCPrev = 0.000000001

                        if(DCPrev == 0):
                            DCPrev = 0.000000001

                        if(isi == 0):
                            isi = 0.000000001

                        if(bui == 0):
                            bui = 0.000000001

                        if(fwix == 0):
                            fwix = 0.000000001
                    except:
                        pass



                    FFMCPrev = FWI.FFMC(TEMP,RH,WIND,RAIN,FFMCPrev)
                    if(FFMCPrev == 0):
                        FFMCPrev = 0.000000001

                    DMCPrev = FWI.DMC(TEMP,RH,RAIN,DMCPrev,LAT,int(MONTH))
                    if(DMCPrev == 0):
                        DMCPrev = 0.000000001


                    DCPrev = FWI.DC(TEMP,RAIN,DCPrev,LAT,int(MONTH))
                    if(DCPrev == 0):
                        DCPrev = 0.000000001

                    isi = FWI.ISI(WIND, FFMCPrev)
                    if(isi == 0):
                        isi = 0.000000001

                    bui = FWI.BUI(DMCPrev,DCPrev)

                    if(bui == 0):
                        bui = 0.000000001
                        
                    fwix = FWI.FWI(isi, bui)

                    if(fwix == 0):
                        fwix = 0.000000001

                    print("RAIN: ", RAIN)
                    print("FFMC: ", FFMCPrev)
                    print("DMC: ", DMCPrev)
                    print("DC: ", DCPrev)
                    print("ISI: ", isi)
                    print("BUI: ", bui)
                    print("FWI: ", fwix)

                    fwix_values[i] = fwix
                    ffmc_values[i] = FFMCPrev
                    dmc_values[i] = DMCPrev
                    dc_values[i] = DCPrev
                    isi_values[i] = isi
                    bui_values[i] = bui
            else:
                print(f"Initial date {initial_date} is not within the range of dates in the DataFrame.")
                break


    dfCurr['fwix'] = fwix_values
    dfCurr['ffmc'] = ffmc_values
    dfCurr['dmc'] = dmc_values
    dfCurr['dc'] = dc_values
    dfCurr['isi'] = isi_values
    dfCurr['bui'] = bui_values


    dfCurr.to_csv(f'OUTPUTNOFIREFWI/{fogos[j]}', index=False)